In [1]:
# GPU 확인
!nvidia-smi

Thu Oct  2 03:46:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os
os.environ['VLLM_USE_V1'] = '0'

In [3]:
# vLLM 설치
!pip install vllm -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.0/15.0 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 MB 8.6 M

In [4]:
# 메모리 정리
import torch
import gc

torch.cuda.empty_cache()
gc.collect()

30

In [5]:
# 모델 로딩

from vllm import LLM, SamplingParams
from transformers import AutoTokenizer, AutoModelForCausalLM
import time

model_name = "facebook/opt-125m"
print(f"\n[모델 로딩 시작] {model_name}")
print("-" * 80)

print("\n1. vLLM 모델 로드... ")
print("-" * 80)

start_vllm_load = time.time()
llm = LLM(model=model_name)
vllm_load_time = time.time() - start_vllm_load

print(f"   vLLM 로드 시간: {vllm_load_time:.2f}초")
print("-" * 80)

print("\n2. HuggingFace 모델 로드 중...")
print("-" * 80)

start_hf_load = time.time()
tokenizer = AutoTokenizer.from_pretrained(model_name)
hf_model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")
hf_load_time = time.time() - start_hf_load

print(f"   HuggingFace 로드 시간: {hf_load_time:.2f}초")
print("-" * 80)
print(f">> vLLM의 초기 로드 시간이 HuggingFace보다 긴 것이 일반적:  KV 캐시 블록 풀 메모리 사전 할당, CUDA 커널들을 사전 컴파일하고 최적화, 배치 처리 준비")
print(f">> HuggingFace는 모델만 로드하고 추론 시점에 필요한 만큼만 메모리를 할당하여 초기 로딩 속도가 더 빠름")



INFO 10-02 03:48:59 [__init__.py:216] Automatically detected platform cuda.

[모델 로딩 시작] facebook/opt-125m
--------------------------------------------------------------------------------

1. vLLM 모델 로드... 
--------------------------------------------------------------------------------
INFO 10-02 03:49:06 [utils.py:328] non-default args: {'disable_log_stats': True, 'model': 'facebook/opt-125m'}


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

INFO 10-02 03:49:44 [__init__.py:742] Resolved architecture: OPTForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


...

INFO 10-02 03:50:05 [default_loader.py:268] Loading weights took 0.35 seconds
INFO 10-02 03:50:06 [model_runner.py:1083] Model loading took 0.2389 GiB and 12.250036 seconds
INFO 10-02 03:50:07 [worker.py:290] Memory profiling takes 1.15 seconds
INFO 10-02 03:50:07 [worker.py:290] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 10-02 03:50:07 [worker.py:290] model weights take 0.24GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 0.47GiB; the rest of the memory reserved for KV Cache is 12.53GiB.
INFO 10-02 03:50:08 [executor_base.py:114] # cuda blocks: 22813, # CPU blocks: 7281
INFO 10-02 03:50:08 [executor_base.py:119] Maximum concurrency for 2048 tokens per request: 178.23x
INFO 10-02 03:50:12 [model_runner.py:1355] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' i

Capturing CUDA graph shapes:   0%|          | 0/35 [00:00<?, ?it/s]

INFO 10-02 03:50:53 [model_runner.py:1507] Graph capturing finished in 41 secs, took 0.14 GiB
INFO 10-02 03:50:53 [worker.py:467] Free memory on device (14.64/14.74 GiB) on startup. Desired GPU memory utilization is (0.9, 13.27 GiB). Actual usage is 0.24 GiB for weight, 0.47 GiB for peak activation, 0.03 GiB for non-torch memory, and 0.14 GiB for CUDAGraph memory. Replace gpu_memory_utilization config with `--kv-cache-memory=13152064614` to fit into requested memory, or `--kv-cache-memory=14626958848` to fully utilize gpu memory. Current kv cache memory in use is 13456151654 bytes.
INFO 10-02 03:50:53 [llm_engine.py:420] init engine (profile, create kv cache, warmup model) took 47.48 seconds
INFO 10-02 03:50:54 [llm.py:295] Supported_tasks: ['generate']
INFO 10-02 03:50:54 [__init__.py:36] No IOProcessor plugins requested by the model
   vLLM 로드 시간: 107.99초
--------------------------------------------------------------------------------

2. HuggingFace 모델 로드 중...
----------------------

model.safetensors:   0%|          | 0.00/251M [00:00<?, ?B/s]

   HuggingFace 로드 시간: 9.01초
--------------------------------------------------------------------------------
>> vLLM의 초기 로드 시간이 HuggingFace보다 긴 것이 일반적:  KV 캐시 블록 풀 메모리 사전 할당, CUDA 커널들을 사전 컴파일하고 최적화, 배치 처리 준비
>> HuggingFace는 모델만 로드하고 추론 시점에 필요한 만큼만 메모리를 할당하여 초기 로딩 속도가 더 빠름


In [6]:
print("1. 단일 추론 속도 비교")
print("=" * 80)

prompt = "The future of AI is"
params = SamplingParams(temperature=0.8, max_tokens=50)

# vLLM
print("\n[vLLM 추론]")
print("-" * 80)

start_vllm = time.time()
vllm_outputs = llm.generate([prompt], params)
vllm_time = time.time() - start_vllm

print(f">> 입력: {prompt}")
print(f">> 출력: {vllm_outputs[0].outputs[0].text[:100]}...")
print(f">> 처리 시간: {vllm_time:.3f}초")


# HuggingFace
print("\n[HuggingFace 추론]")
print("-" * 80)

start_hf = time.time()
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
with torch.no_grad():
    hf_outputs = hf_model.generate(
        **inputs,
        max_new_tokens=50,
        temperature=0.8,
        do_sample=True
    )
hf_time = time.time() - start_hf
hf_text = tokenizer.decode(hf_outputs[0], skip_special_tokens=True)

print(f">> 입력: {prompt}")
print(f">> 출력: {hf_text[len(prompt):len(prompt)+100]}...")
print(f">> 처리 시간: {hf_time:.3f}초")

print(f"\n>> 속도 비교: vLLM이 HuggingFace보다 {hf_time/vllm_time:.1f}배 빠름")

1. 단일 추론 속도 비교

[vLLM 추론]
--------------------------------------------------------------------------------


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

>> 입력: The future of AI is
>> 출력:  quite bright. From how it interacts with physical objects to how people interact with them, AI will...
>> 처리 시간: 2.902초

[HuggingFace 추론]
--------------------------------------------------------------------------------
>> 입력: The future of AI is
>> 출력:  now
The future of AI is now. The technology that we use today will change forever. Our world has ch...
>> 처리 시간: 1.269초

>> 속도 비교: vLLM이 HuggingFace보다 0.4배 빠름


In [7]:
print("2. 배치 처리 속도 비교 (Continuous Batching)")
print("=" * 80)

prompts = [
    "Artificial intelligence can",
    "Machine learning is",
    "Deep learning helps",
    "Neural networks are",
    "Python programming"
]

# vLLM
print(f"\n[vLLM 배치 추론] {len(prompts)}개 동시 처리")
print("-" * 80)

start_vllm_batch = time.time()
vllm_batch_outputs = llm.generate(prompts, params)
vllm_batch_time = time.time() - start_vllm_batch

for i, output in enumerate(vllm_batch_outputs[:5]):
    print(f"  [{i+1}] {prompts[i]}")
    print(f"      -> {output.outputs[0].text[:60]}...")

print(f"  ... (총 {len(prompts)}개)")
print(f">> 총 처리 시간: {vllm_batch_time:.3f}초")
print(f">> 처리량: {len(prompts)/vllm_batch_time:.2f} 요청/초")
print("-" * 80)

# HuggingFace
print(f"\n[HuggingFace 배치 추론] {len(prompts)}개 순차 처리")
print("-" * 80)

start_hf_batch = time.time()
hf_batch_outputs = []
for prompt_item in prompts:
    inputs = tokenizer(prompt_item, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = hf_model.generate(
            **inputs,
            max_new_tokens=50,
            temperature=0.8,
            do_sample=True
        )
    hf_batch_outputs.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
hf_batch_time = time.time() - start_hf_batch

for i in range(5):
    print(f"  [{i+1}] {prompts[i]}")
    print(f"      -> {hf_batch_outputs[i][len(prompts[i]):len(prompts[i])+60]}...")

print(f"  ... (총 {len(prompts)}개)")
print(f">> 총 처리 시간: {hf_batch_time:.3f}초")
print(f">> 처리량: {len(prompts)/hf_batch_time:.2f} 요청/초")

print(f"\n>> 배치 처리 속도 비교: vLLM이 HuggingFace보다 {hf_batch_time/vllm_batch_time:.1f}배 빠름")

2. 배치 처리 속도 비교 (Continuous Batching)

[vLLM 배치 추론] 5개 동시 처리
--------------------------------------------------------------------------------


Adding requests:   0%|          | 0/5 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/5 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

  [1] Artificial intelligence can
      ->  be used to make a simple smart cell
Researchers at the Univ...
  [2] Machine learning is
      ->  the fastest way to change the world. If you don’t have the ...
  [3] Deep learning helps
      ->  an emotional 'blind eye' to how to improve social interacti...
  [4] Neural networks are
      ->  an integral part of all current scientific and technologica...
  [5] Python programming
      ->  sounds like fun, though.  I had the trouble putting togethe...
  ... (총 5개)
>> 총 처리 시간: 0.281초
>> 처리량: 17.81 요청/초
--------------------------------------------------------------------------------

[HuggingFace 배치 추론] 5개 순차 처리
--------------------------------------------------------------------------------
  [1] Artificial intelligence can
      ->  take on a whole new form.

The future of artificial intelli...
  [2] Machine learning is
      ->  the future of the industry. It will be a race against time....
  [3] Deep learning helps
      ->  for human lea

In [8]:
print("3. 다중 샘플링 처리 속도 비교")
print("=" * 80)

sampling_prompts = [
    "The future of artificial intelligence",
    "Climate change solutions include",
    "Quantum computing will revolutionize",
    "Renewable energy sources are",
    "Space exploration enables"
]

print(f"\n{len(sampling_prompts)}개의 프롬프트 배치 처리 테스트")

# vLLM
print("\n[vLLM 배치 샘플링 방식]")
print("-" * 80)

start_vllm_sampling = time.time()

params = SamplingParams(temperature=0.7, max_tokens=30)

vllm_outputs = llm.generate(sampling_prompts, params)

vllm_sampling_time = time.time() - start_vllm_sampling

print(f"샘플링 결과:")
for i in range(min(3, len(sampling_prompts))):
    print(f"  {sampling_prompts[i][:30]}...")
    print(f"    -> {vllm_outputs[i].outputs[0].text[:50]}...")

print(f"\n>> 총 처리 시간: {vllm_sampling_time:.3f}초 ({len(sampling_prompts)}개 요청)")
print("-" * 80)

# HuggingFace
print("\n[HuggingFace 순차 샘플링]")
print("-" * 80)

start_hf_sampling = time.time()

hf_outputs = []

for prompt in sampling_prompts:
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        output = hf_model.generate(
            **inputs, max_new_tokens=30, temperature=0.7, do_sample=True
        )
        hf_outputs.append(tokenizer.decode(output[0], skip_special_tokens=True))

hf_sampling_time = time.time() - start_hf_sampling

print(f"샘플링 결과:")
for i in range(min(3, len(sampling_prompts))):
    print(f"  {sampling_prompts[i][:30]}...")
    print(f"    -> {hf_outputs[i][len(sampling_prompts[i]):len(sampling_prompts[i])+50]}...")

print(f"\n>> 총 처리 시간: {hf_sampling_time:.3f}초 ({len(sampling_prompts)}개 요청)")

print(f"\n>> 다중 샘플링 속도 비교: vLLM이 HuggingFace보다 {hf_sampling_time/vllm_sampling_time:.1f}배 빠름")

3. 다중 샘플링 처리 속도 비교

5개의 프롬프트로 각각 다른 temperature 설정 테스트

[vLLM 배치 샘플링 방식]
--------------------------------------------------------------------------------


Adding requests:   0%|          | 0/5 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/5 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/5 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/5 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

temperature=0.1 샘플링 (temperature=0.1):
  The future of artificial intel...
    ->  is in the hands of the people.

The future of art...
  Climate change solutions inclu...
    ->  a new approach to the problem of climate change.
...
	emperature=1 샘플링 (temperature=1.0):
  The future of artificial intel...
    -> 
This is not how robots make money any more.
First...
  Climate change solutions inclu...
    ->  science
Now that you have described it, my first ...

>> 총 처리 시간: 0.403초 (10개 요청)
--------------------------------------------------------------------------------

[HuggingFace 순차 샘플링]
--------------------------------------------------------------------------------
temperature=0.1 (temperature=0.1):
  The future of artificial intel...
    ->  is in the hands of the people.

The future of art...
  Climate change solutions inclu...
    ->  a new approach to the problem of climate change.
...

temperature=1.0 (temperature=1.0):
  The future of artificial intel...
    ->  should be more

In [9]:
print("4. 메모리 효율성 비교 (PagedAttention): 긴 시퀀스 생성 시 메모리 사용량")
print("-" * 80)

test_prompt = "Write a detailed story about artificial intelligence:"
max_tokens = 2000

print(f"\n최대 생성 토큰: {max_tokens}개")
print("─" * 60)

# vLLM 메모리 측정
torch.cuda.empty_cache()
gc.collect()
torch.cuda.reset_peak_memory_stats()

memory_before_vllm = torch.cuda.memory_allocated() / (1024**2)

params = SamplingParams(temperature=0.7, max_tokens=max_tokens)
start_vllm = time.time()
vllm_output = llm.generate([test_prompt], params)
vllm_time = time.time() - start_vllm

memory_after_vllm = torch.cuda.memory_allocated() / (1024**2)
memory_peak_vllm = torch.cuda.max_memory_allocated() / (1024**2)
memory_used_vllm = memory_peak_vllm - memory_before_vllm

print(f"\nvLLM PagedAttention:")
print("-" * 80)
print(f"  처리 시간: {vllm_time:.3f}초")
print(f"  추론 전 메모리: {memory_before_vllm:.2f} MB")
print(f"  최대 메모리: {memory_peak_vllm:.2f} MB")
print(f"  추가 사용: {memory_used_vllm:.2f} MB")

# HuggingFace 메모리 측정
torch.cuda.empty_cache()
gc.collect()
torch.cuda.reset_peak_memory_stats()

memory_before_hf = torch.cuda.memory_allocated() / (1024**2)

inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")
start_hf = time.time()
with torch.no_grad():
    hf_output = hf_model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=0.7,
        do_sample=True
    )
hf_time = time.time() - start_hf

memory_after_hf = torch.cuda.memory_allocated() / (1024**2)
memory_peak_hf = torch.cuda.max_memory_allocated() / (1024**2)
memory_used_hf = memory_peak_hf - memory_before_hf

print(f"\nHuggingFace:")
print("-" * 80)
print(f"  처리 시간: {hf_time:.3f}초")
print(f"  추론 전 메모리: {memory_before_hf:.2f} MB")
print(f"  최대 메모리: {memory_peak_hf:.2f} MB")
print(f"  추가 사용: {memory_used_hf:.2f} MB")

# 비교
memory_savings = memory_used_hf - memory_used_vllm
memory_savings_pct = (memory_savings / memory_used_hf) * 100
speed_ratio = hf_time / vllm_time

print("\n" + "─" * 60)
print("비교 결과:")
print(f"  메모리 절약: {memory_savings:.2f} MB ({memory_savings_pct:.1f}%)")
print(f"  속도: vLLM이 {speed_ratio:.1f}배 빠름")
print("─" * 60)


print("\n vLLM PagedAttention:")
print("  - KV 캐시를 고정 크기 블록(페이지)으로 분할")
print("  - 필요할 때마다 블록을 동적으로 할당, 메모리 단편화 최소화 및 효율적 재사용")

print("\n HuggingFace:")
print("  - KV 캐시를 연속된 메모리 공간에 사전 할당, 최대 시퀀스 길이만큼 메모리 예약")

4. 메모리 효율성 비교 (PagedAttention): 긴 시퀀스 생성 시 메모리 사용량
--------------------------------------------------------------------------------

최대 생성 토큰: 2000개
────────────────────────────────────────────────────────────


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


vLLM PagedAttention:
--------------------------------------------------------------------------------
  처리 시간: 5.818초
  추론 전 메모리: 13580.13 MB
  최대 메모리: 13581.25 MB
  추가 사용: 1.12 MB

HuggingFace:
--------------------------------------------------------------------------------
  처리 시간: 4.262초
  추론 전 메모리: 13580.13 MB
  최대 메모리: 13626.64 MB
  추가 사용: 46.50 MB

────────────────────────────────────────────────────────────
비교 결과:
  메모리 절약: 45.38 MB (97.6%)
  속도: vLLM이 0.7배 빠름
────────────────────────────────────────────────────────────

 vLLM PagedAttention:
  - KV 캐시를 고정 크기 블록(페이지)으로 분할
  - 필요할 때마다 블록을 동적으로 할당, 메모리 단편화 최소화 및 효율적 재사용

 HuggingFace:
  - KV 캐시를 연속된 메모리 공간에 사전 할당, 최대 시퀀스 길이만큼 메모리 예약


In [10]:
print("5. 동시 사용자 처리 비교")
print("=" * 80)

user_queries = [f"User {i}: Hello, my name is" for i in range(1, 11)]

# vLLM 동시 처리
print(f"\n[vLLM] 10명의 동시 사용자 요청 처리 (Continuous Batching)")
print("-" * 80)
start_vllm_users = time.time()
vllm_user_outputs = llm.generate(user_queries, SamplingParams(temperature=0.7, max_tokens=20))
vllm_users_time = time.time() - start_vllm_users

for i in range(3):
    print(f"  {user_queries[i]}")
    print(f"    -> {vllm_user_outputs[i].outputs[0].text}")
print(f"  ... (총 10개 요청)")
print(f">> 처리 시간: {vllm_users_time:.3f}초")
print(f">> 처리 능력: {10/vllm_users_time:.1f} 사용자/초")
print("-" * 80)

# HuggingFace 순차 처리
print(f"\n[HuggingFace] 10명의 사용자 요청 순차 처리")
print("-" * 80)
start_hf_users = time.time()
hf_user_outputs = []
for query in user_queries:
    inputs = tokenizer(query, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = hf_model.generate(
            **inputs,
            max_new_tokens=20,
            temperature=0.7,
            do_sample=True
        )
    hf_user_outputs.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
hf_users_time = time.time() - start_hf_users

for i in range(3):
    print(f"  {user_queries[i]}")
    print(f"    -> {hf_user_outputs[i][len(user_queries[i]):]}")
print(f"  ... (총 10개 요청)")
print(f">> 처리 시간: {hf_users_time:.3f}초")
print(f">> 처리 능력: {10/hf_users_time:.1f} 사용자/초")

print(f"\n동시 사용자 처리: vLLM이 HuggingFace보다 {hf_users_time/vllm_users_time:.1f}배 빠름")
print("\nvLLM의 Continuous Batching:")
print("  - 여러 사용자 요청을 효율적으로 동시 처리, 실시간 서비스에 최적화, 높은 Throughput")
print("\nHuggingFace:")
print("  - 기본적으로 순차 처리, 배치 처리도 가능하지만 vLLM만큼 최적화되지 않음")

5. 동시 사용자 처리 비교

[vLLM] 10명의 동시 사용자 요청 처리 (Continuous Batching)
--------------------------------------------------------------------------------


Adding requests:   0%|          | 0/10 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/10 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

  User 1: Hello, my name is
    ->  Mark and I am an adult dog owner living in Texas. I’m a little older than
  User 2: Hello, my name is
    ->  Yurick. I am a new player here. I am a moderator of /r/t
  User 3: Hello, my name is
    ->  Carla and I'm from Southern California and I'm looking for an apartment in a quiet area of
  ... (총 10개 요청)
>> 처리 시간: 0.163초
>> 처리 능력: 61.4 사용자/초
--------------------------------------------------------------------------------

[HuggingFace] 10명의 사용자 요청 순차 처리
--------------------------------------------------------------------------------
  User 1: Hello, my name is
    ->  Nisha and I'm a student at university, I'm looking for a job in the IT sector
  User 2: Hello, my name is
    ->  Ygritte. I am a child of the gods, and I am a little girl.
  User 3: Hello, my name is
    ->  Andrew W. and I am a software developer who loves to make awesome mobile apps. I am currently
  ... (총 10개 요청)
>> 처리 시간: 2.152초
>> 처리 능력: 4.6 사용자/초

동시 사용자 처리: vLLM이 HuggingFa